In [1]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup as bs

import src.cleaning as cleaning

In [2]:
trucks = pd.read_csv('data/truck_transfers.csv')
april = pd.read_csv('data/april_2014.csv')

In [3]:
stations = cleaning.stations_coordinates (april)
# stations capacity
trucks = cleaning.coordinate_columns (trucks)
trucks = cleaning.truck_trips_coordinates (stations, trucks)
trucks = cleaning.time_difference (trucks)

In [4]:
trucks = trucks[trucks['time_difference'] > '0 days']

In [9]:
def time_difference (df):
    df['started_at'] = pd.to_datetime(df['started_at'], infer_datetime_format = True)
    df['ended_at'] = pd.to_datetime(df['ended_at'], infer_datetime_format = True)
    return df

In [10]:
df = pd.read_csv('data/april_2014.csv')
df = time_difference (df)

In [13]:

# Calculate the capacity of each station
stations = set(df['start_station_id']).union(set(df['end_station_id']))
station_capacity = {}
for station_id in stations:
    num_starts = len(df[df['start_station_id'] == station_id])
    num_ends = len(df[df['end_station_id'] == station_id])
    capacity = num_starts + num_ends
    station_capacity[station_id] = capacity

# Find out how many bikes are in each station at a given time
start_time = pd.Timestamp('2014-04-01 00:00:00')
end_time = pd.Timestamp('2014-04-30 23:59:59')
time_filter = (df['started_at'] >= start_time) & (df['ended_at'] <= end_time)
filtered_df = df[time_filter]
bike_counts = filtered_df.groupby(['start_station_id', 'end_station_id']).size().reset_index(name='count')

bike_counts.sort_values(by=['count'], ascending=False)

,start_station_id,end_station_id,count
63172,2006,2006,990
53253,499,499,627
13720,281,281,514
20093,318,477,402
38998,432,293,323
...,...,...,...
8752,247,430,1
20153,318,2002,1
20164,318,3002,1
20168,319,116,1


In [14]:
# Merge capacity and bike count dataframes
station_info = pd.DataFrame.from_dict(station_capacity, orient='index', columns=['capacity'])
station_info.reset_index(inplace=True)
station_info = station_info.rename(columns={'index': 'station id'})
station_info = station_info.merge(bike_counts, on=['start_station_id', 'end_station_id'], how='left')
station_info['count'] = station_info['count'].fillna(0)

# Print the resulting dataframe
print(station_info.head())

KeyError: 'start_station_id'

In [49]:
last_end = april[april['bike_id'] == 21062]['end_station_id'].to_list()[:-1]
etime = april[april['bike_id'] == 21062]['ended_at'].to_list()[:-1]

next_start = april[april['bike_id'] == 21062]['start_station_id'].to_list()[1:]
stime = april[april['bike_id'] == 21062]['started_at'].to_list()[1:]

trip_kind = ['human']*len(stime)

In [55]:
some_df = pd.DataFrame({'last_end': last_end, 'end_time': etime, 'next_start': next_start, 'start_time': stime, 'trip_kind': trip_kind})

def time_difference (df):
    df['start_time'] = pd.to_datetime(df['start_time'], infer_datetime_format = True)
    df['end_time'] = pd.to_datetime(df['end_time'], infer_datetime_format = True)
    return df

some_df = time_difference(some_df)

some_df['time_difference'] = some_df['start_time'] - some_df['end_time']

In [56]:
some_df

,last_end,end_time,next_start,start_time,trip_kind,time_difference
0,2008,2014-04-01 00:09:25,2008,2014-04-01 07:41:30,human,0 days 07:32:05
1,224,2014-04-01 07:48:08,224,2014-04-01 08:54:32,human,0 days 01:06:24
2,360,2014-04-01 08:58:20,306,2014-04-02 15:25:04,human,1 days 06:26:44
3,147,2014-04-02 15:32:24,147,2014-04-02 17:22:38,human,0 days 01:50:14
4,463,2014-04-02 17:38:33,463,2014-04-02 17:38:49,human,0 days 00:00:16
...,...,...,...,...,...,...
136,521,2014-04-28 21:33:53,521,2014-04-28 22:34:44,human,0 days 01:00:51
137,2021,2014-04-28 22:43:36,2021,2014-04-28 22:50:15,human,0 days 00:06:39
138,423,2014-04-28 22:55:07,423,2014-04-29 06:35:38,human,0 days 07:40:31
139,474,2014-04-29 06:48:51,474,2014-04-29 07:20:50,human,0 days 00:31:59


In [60]:

for i, row in some_df.iterrows():
    if row['last_end'] != row['next_start']:
        last_end.append(row['next_start'])
        next_start.append(row['last_end'])
        stime.append(row['end_time'] + 1/3 * row['time_difference'])
        etime.append(row['start_time'] + 2/3 * row['time_difference'])
        trip_kind.append('truck')

In [61]:
some_df = pd.DataFrame({'last_end': last_end, 'end_time': etime, 'next_start': next_start, 'start_time': stime, 'trip_kind': trip_kind})
some_df

,last_end,end_time,next_start,start_time,trip_kind
0,2008,2014-04-01 00:09:25,2008,2014-04-01 07:41:30,human
1,224,2014-04-01 07:48:08,224,2014-04-01 08:54:32,human
2,360,2014-04-01 08:58:20,306,2014-04-02 15:25:04,human
3,147,2014-04-02 15:32:24,147,2014-04-02 17:22:38,human
4,463,2014-04-02 17:38:33,463,2014-04-02 17:38:49,human
...,...,...,...,...,...
151,430,2014-04-25 12:21:02,310,2014-04-23 21:36:02,truck
152,317,2014-04-25 08:25:37,293,2014-04-25 06:58:17,truck
153,493,2014-04-25 17:00:18.333333333,546,2014-04-25 16:02:31.666666666,truck
154,306,2014-04-28 04:49:33.333333333,315,2014-04-26 20:51:58.666666666,truck


In [66]:
some_df = time_difference(some_df)
some_df.sort_values(by=['end_time'], ascending=True, inplace=True)
some_df.head(30)

,last_end,end_time,next_start,start_time,trip_kind
0,2008,2014-04-01 00:09:25.000000000,2008,2014-04-01 07:41:30.000000000,human
1,224,2014-04-01 07:48:08.000000000,224,2014-04-01 08:54:32.000000000,human
2,360,2014-04-01 08:58:20.000000000,306,2014-04-02 15:25:04.000000000,human
3,147,2014-04-02 15:32:24.000000000,147,2014-04-02 17:22:38.000000000,human
4,463,2014-04-02 17:38:33.000000000,463,2014-04-02 17:38:49.000000000,human
5,368,2014-04-02 17:46:22.000000000,368,2014-04-02 19:52:41.000000000,human
6,341,2014-04-02 20:08:13.000000000,341,2014-04-03 12:34:19.000000000,human
141,306,2014-04-03 11:42:53.333333333,360,2014-04-01 19:07:14.666666666,truck
7,473,2014-04-03 12:46:45.000000000,522,2014-04-05 11:56:28.000000000,human
8,540,2014-04-05 12:08:32.000000000,540,2014-04-05 12:11:03.000000000,human
